# Necessary Libraries

In [2]:
import pandas as pd
import mysql.connector as mysql
import kagglehub
from kagglehub import KaggleDatasetAdapter
from mysql.connector import Error

KeyboardInterrupt: 

# Create Schema

In [ ]:
# %run Scripts/schema_creation.py

✅ Successfully connected to MySQL database
Database create or already exists
Database schema created successfully


# Wildfire Section

In [ ]:
use_cols = ['OBJECTID', 'FIRE_NAME', 'DISCOVERY_DATE', 'NWCG_GENERAL_CAUSE', 'FIRE_SIZE', 'STATE', 'FIPS_NAME']

# Download latest version
wildfire_df = kagglehub.load_dataset(handle = "behroozsohrabi/us-wildfire-records-6th-edition", path = "data.csv", 
                                   adapter = KaggleDatasetAdapter.PANDAS, 
                                   pandas_kwargs={"usecols": use_cols, "compression": "zip"})


# Filter states that we're keeping

states_to_keep = ['CA', 'TX', 'GA', 'FL', 'AZ']

wildfire_df.loc[~wildfire_df['STATE'].isin(states_to_keep), :] = None

wildfire_df.dropna(inplace = True)


# Convert DISCOVERY_DATE to datetime object to remove records prior to year 2000

wildfire_df['DISCOVERY_DATE'] = pd.to_datetime(wildfire_df['DISCOVERY_DATE'], format = ('%m/%d/%Y'))

wildfire_df = wildfire_df[wildfire_df['DISCOVERY_DATE'].dt.year > 2000]


# Change to int for compatiabilty with Primary Key

wildfire_df['OBJECTID'] = wildfire_df['OBJECTID'].astype(int)


# Rename some columns for clarity

wildfire_df = wildfire_df.rename(columns = {"NWCG_GENERAL_CAUSE": "SPECIFIC_CAUSE", "FIPS_NAME": "COUNTY"})


# Call script to map state FIPS codes to new columns

%run Scripts/mapping.py

wildfire_df['state_id'] = wildfire_df['STATE'].map(state_fips_mapping)


# Realign columns and load location data to prep for county_id mapping

wildfire_df.rename(columns = {'COUNTY': 'county_name'}, inplace = True)

locations_df = pd.read_csv("data/locations.csv")


# Map county codes to wildfire data

wildfire_df = match_county_id(wildfire_df, locations_df)


# Use missing values scripts to fill anything missing

%run Scripts/missing_values.py

missing_dict = {
    "FL": 86,
    "GA": 29
}

wildfire_df = fill_missing_values(wildfire_df, missing_dict, "STATE", "county_id")


# Connect to database

%run Scripts/connect.py


cursor = conn.cursor()

columns = wildfire_df.columns

data_to_insert = list(wildfire_df[columns].itertuples(index = False, name = None))

clause = """
INSERT IGNORE INTO wildfire (fire_id, fire_name, discovery_date, cause, fire_size, state_name, county_name, state_id, county_id)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

batch_size = 10000
start_index = 0
try:
    for i in range(start_index, len(data_to_insert), batch_size):
        batch = data_to_insert[i:i+batch_size]
        cursor.executemany(clause, batch)
        conn.commit()
        print('Data loaded successfully')
except Error as err:
    print(f"Data unable to be loaded: {err}")

cursor.close()
conn.close()

# Housing Data

In [17]:
# Download Zillow data

"""df = pd.read_csv("/your path.csv")"""


# To parquet if needed for on-site storage or to load to github

"""df.to_parquet("/your path.parquet)"""


# Load and remove columns that aren't used

bottom_tier_housing = pd.read_parquet('data/bottom_housing.parquet')
top_tier_housing = pd.read_parquet('data/top_housing.parquet')

remove_cols = ['RegionID', 'RegionType', 'StateName', 'Metro', 'SizeRank']

bottom_tier_housing.drop(columns = remove_cols, inplace = True)
top_tier_housing.drop(columns = remove_cols, inplace = True)


# Filter out states not using and remove na values

states = ['CA', 'TX', 'GA', 'FL', 'AZ']

bottom_tier_housing.loc[~bottom_tier_housing['State'].isin(states), :] = None
top_tier_housing.loc[~top_tier_housing['State'].isin(states), :] = None

bottom_tier_housing.dropna(inplace = True)
top_tier_housing.dropna(inplace = True)


# Combine the data frames and pivot long the date features and price values and change date column to date dtype

housing_df = pd.concat([bottom_tier_housing, top_tier_housing], axis = 0, ignore_index = True)

housing_df = housing_df.melt(id_vars = ['RegionName', 'State', 'CountyName'], var_name = 'Date', value_name = 'Price')
housing_df['Date'] = pd.to_datetime(housing_df['Date']).dt.date


# Map state and county id codes 

%run Scripts/mapping.py

housing_df['state_id'] = housing_df['State'].map(state_fips_mapping)

housing_df.rename(columns = {'CountyName': 'county_name'}, inplace = True)

locations_df = pd.read_csv("data/locations.csv")

housing_df = match_county_id(housing_df, locations_df)


# Connect and load data

%run Scripts/connect.py

cursor = conn.cursor()

columns = housing_df.columns

data_to_insert = list(housing_df[columns].itertuples(index = False, name = None))

clause = """
INSERT IGNORE INTO housing (region_name, state_name, county_name, assessment_date, price, state_id, county_id)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

batch_size = 15000
start_index = 0

try:
    for i in range(start_index, len(data_to_insert), batch_size):
        batch = data_to_insert[i:i + batch_size]
        
        try:
            cursor.executemany(clause, batch)
            conn.commit()
            print(f"Data loaded successfully for batch {i // batch_size + 1}")
        except Error as e:
            if e == '2013 (HY000): Lost connection to MySQL server during query':
                print("Lost connection, attempting to reconnect...")
                reconnect()
                cursor.executemany(clause, batch)  # Retry query
                conn.commit()

except Error as err:
    print(f"Error: {err}")

cursor.close()
conn.close()

100%|██████████| 994800/994800 [01:01<00:00, 16259.47it/s]


✅ Successfully connected to MySQL database
Data loaded successfully for batch 1
Data loaded successfully for batch 2
Data loaded successfully for batch 3
Data loaded successfully for batch 4
Data loaded successfully for batch 5
Data loaded successfully for batch 6
Data loaded successfully for batch 7
Data loaded successfully for batch 8
Data loaded successfully for batch 9
Data loaded successfully for batch 10


# Rental Data

In [ ]:
# Read in rental data

rent_index = pd.read_csv("data/Observed Rent Index by City.csv")


# Remove unused columns and filter out states

remove_cols = ['RegionID', 'RegionType', 'StateName', 'Metro', 'SizeRank']
states = ['CA', 'TX', 'GA', 'FL', 'AZ']

rent_index.drop(columns = remove_cols, inplace = True)
rent_index.loc[~rent_index['State'].isin(states), :] = None

rent_index.dropna(inplace = True)


# Pivot long date features and price values and change date column to date dtype

rent_index = rent_index.melt(id_vars = ['RegionName', 'State', 'CountyName'], var_name = 'Date', value_name = 'Price')
rent_index['Date'] = pd.to_datetime(housing_df['Date']).dt.date


# Map state and county id codes 

%run Scripts/mapping.py

rent_index['state_id'] = rent_index['State'].map(state_fips_mapping)

rent_index.rename(columns = {'CountyName': 'county_name'}, inplace = True)

locations_df = pd.read_csv("data/locations.csv")

rent_index = match_county_id(rent_index, locations_df)


# Connect and load rental data into database

%run Scripts/connect.py

cursor = conn.cursor()

columns = rent_index.columns

data_to_insert = list(rent_index[columns].itertuples(index = False, name = None))

clause = """
INSERT IGNORE INTO rentals (region_name, state_name, county_name, assessment_date, price, state_id, county_id)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

batch_size = 15000

try:
    for i in range(0, len(data_to_insert), batch_size):
        batch = data_to_insert[i:i+batch_size]

        try:
            cursor.executemany(clause, batch)
            conn.commit()
            print(f"Data loaded successfully for batch {i // batch_size + 1}")
        except Error as e:
            if e == "2013 (HY000): Lost connection to MySQL server during query":
                print("Lost connection, attempting to reconnect...")
                reconnect()
                cursor.executemany(clause, batch)
                conn.commmit()
except Error as err:
    print(err)


cursor.close()
conn.close()

# Location and Population Data

In [16]:
# Load location data from census website

ca_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st06_ca_place_by_county2020.txt', delimiter = '|')
tx_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st48_tx_place_by_county2020.txt', delimiter = '|')
ga_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st13_ga_place_by_county2020.txt', delimiter = '|')
fl_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st12_fl_place_by_county2020.txt', delimiter = '|')
az_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st04_az_place_by_county2020.txt', delimiter = '|')


# Combine data

combined_df = pd.concat([ca_codes, tx_codes, ga_codes, fl_codes, az_codes], axis = 0, ignore_index=True)


#Drop unused columns

columns_to_drop = ['PLACENS', 'TYPE', 'CLASSFP', 'FUNCSTAT']

combined_df.drop(columns = columns_to_drop, inplace = True)


# Rename columns for mapping, and load updated data to folder for reference for other tables

combined_df.rename(columns = {'COUNTYNAME': 'county_name', 'COUNTYFP': 'county_id', 'STATEFP': 'state_id'}, inplace = True)

combined_df.to_csv("data/locations.csv")


# Add separate location data into dict for syncing

location_dict = {
    'CA': ca_codes,
    'TX': tx_codes,
    'GA': ga_codes,
    'FL': fl_codes,
    'AZ': az_codes
}

# Load population data from census website

"""state_df = pd.read_csv("/your path.csv")"""


# Add states to dictionary to make transforming easier

state_population_dict = {}

states = ['CA', 'TX', 'GA', 'FL', 'AZ']

for state in states:
    state_population_dict[state] = pd.read_csv(f"data/{state} City population estimates.csv")


# Sync location and population data to standardize state, county and place codes

%run Scripts/sync.py

sync(states, state_population_dict, location_dict)


# Combine state_population_dict for loading

population_df = pd.concat(state_population_dict.values(), ignore_index = True)


# Replace any A values in official Census feature

population_df['CENSUS2010POP'] = population_df['CENSUS2010POP'].apply(lambda x: 0 if x == 'A' else x)


# Connect and load location and population data

%run Scripts/connect.py


# Location loading script

cursor = conn.cursor()

columns = combined_df.columns

data_to_insert = list(combined_df[columns].itertuples(index = False, name = None))

insert_clause = """
INSERT IGNORE INTO locations (state_name, state_id, county_id, county_name, place_id, place_name)
Values (%s, %s, %s, %s, %s, %s)
"""

try:
    cursor.executemany(insert_clause, data_to_insert)
    conn.commit()
    print('Data loaded successfully')
except Error as err:
    print(f"Data unable to be loaded: {err}")



# Population loading script

data_to_insert = list(population_df[['STATE', 'COUNTYFP', 'PLACEFP', 'CENSUS2010POP', 'POPESTIMATE2011', 'POPESTIMATE2012', 
                                     'POPESTIMATE2013', 'POPESTIMATE2014', 'POPESTIMATE2015','POPESTIMATE2016', 'POPESTIMATE2017', 
                                     'POPESTIMATE2018','POPESTIMATE2019']].itertuples(index = False, name = None))

clause = """
INSERT IGNORE INTO census (state_id, county_id, place_id, true_pop_2010, pop_estimate_2011, pop_estimate_2012, pop_estimate_2013, pop_estimate_2014, 
pop_estimate_2015, pop_estimate_2016, pop_estimate_2017, pop_estimate_2018, pop_estimate_2019)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

try:
    cursor.executemany(clause, data_to_insert)
    conn.commit()
    print('Data loaded successfully')
except Error as err:
    print(f"Data unable to be loaded: {err}")

cursor.close()
conn.close()

✅ Successfully connected to MySQL database
Data loaded successfully
Data loaded successfully


# Test Queries

In [44]:
%run Scripts/connect.py

query ="""
SELECT 
    l.county_name, 
    COUNT(w.fire_id) AS total_fires, 
    SUM(w.fire_size) AS total_fire_size
FROM wildfire w
JOIN locations l 
    ON w.state_id = l.state_id 
    AND w.county_id = l.county_id
WHERE YEAR(w.discovery_date) = 2015
GROUP BY l.county_name
ORDER BY total_fires DESC
"""

df = pd.read_sql(query, conn)

conn.close()

✅ Successfully connected to MySQL database


C:\Users\Taylor\AppData\Local\Temp\ipykernel_23384\588213273.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [39]:
%run Scripts/connect.py

query ="""
SELECT 
    l.state_name, 
    YEAR(w.discovery_date) AS fire_year, 
    AVG(w.fire_size) AS avg_total_burned
FROM wildfire w
JOIN locations l 
    ON w.state_id = l.state_id 
GROUP BY l.state_name, YEAR(w.discovery_date)
ORDER BY fire_year ASC, avg_total_burned DESC
"""

df = pd.read_sql(query, conn)

conn.close()

✅ Successfully connected to MySQL database


C:\Users\Taylor\AppData\Local\Temp\ipykernel_23384\438317897.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
